In [ ]:
!pip install PyPDF2



In [ ]:
!pip install spacy



In [ ]:
pip install gensim


In [ ]:
!python -m spacy download es_core_news_sm

In [13]:
import os
import re
import spacy
import nltk
import PyPDF2
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


##LECTURA DE PDFs

In [ ]:
import os
import PyPDF2
from pdf2image import convert_from_path
import pytesseract
from pytesseract import Output

# Ruta del directorio donde están los PDFs
ruta_directorio = "/content/resoluciones/"

# Función para extraer texto de PDFs normales
def extraer_texto_pdf(ruta_archivo):
    texto = ""
    try:
        with open(ruta_archivo, 'rb') as archivo_pdf:
            lector_pdf = PyPDF2.PdfReader(archivo_pdf)
            for pagina in range(len(lector_pdf.pages)):
                texto += lector_pdf.pages[pagina].extract_text() or ""
    except PyPDF2.errors.PdfReadError as e:
        print(f"Error al leer el archivo {ruta_archivo}: {e}")
    except Exception as e:
        print(f"Ocurrió un error al procesar {ruta_archivo}: {e}")
    return texto

# Función para extraer texto de PDFs que contienen imágenes usando OCR
def extraer_texto_ocr(ruta_archivo):
    texto = ""
    try:
        # Convertir las páginas del PDF a imágenes
        paginas_imagen = convert_from_path(ruta_archivo)
        for pagina_imagen in paginas_imagen:
            # Aplicar OCR para extraer el texto de la imagen
            texto += pytesseract.image_to_string(pagina_imagen, lang='spa')  # OCR en español
    except Exception as e:
        print(f"Error al aplicar OCR a {ruta_archivo}: {e}")
    return texto

# Función para procesar PDFs (detectar si tienen texto o son imágenes)
def procesar_pdf(ruta_archivo):
    try:
        texto = extraer_texto_pdf(ruta_archivo)
        
        if not texto.strip():  # Si no hay texto, aplica OCR
            print(f"Aplicando OCR al archivo {ruta_archivo}")
            texto = extraer_texto_ocr(ruta_archivo)
        
        return texto
    
    except Exception as e:
        print(f"Error general al procesar el archivo {ruta_archivo}: {e}")
        return ""

# Obtener una lista de todos los archivos PDF en el directorio
archivos_pdf = [f for f in os.listdir(ruta_directorio) if f.endswith('.pdf')]

# Leer y almacenar el contenido de todos los PDFs (texto o OCR)
resoluciones = []
for archivo in archivos_pdf:
    ruta_completa = os.path.join(ruta_directorio, archivo)
    texto = procesar_pdf(ruta_completa)
    
    if texto:  # Solo agregar si se extrajo algún texto
        resoluciones.append(texto)

# Mostrar el contenido de las primeras resoluciones extraídas
for i, texto in enumerate(resoluciones[:3]):
    print(f"Resolución {i + 1}:\n{texto[:500]}...\n")  # Mostramos los primeros 500 caracteres


##LIMPIEZA DE TEXTO


In [ ]:
# Función para limpiar el texto eliminando números irrelevantes
def limpiar_texto(texto):
    # 1. Convertir a minúsculas
    texto = texto.lower()

    # 2. Eliminar números que no sean años relevantes (por ejemplo, '2007')
    #    - Mantener los años (números de 4 dígitos que empiezan con 19 o 20)
    #    - Eliminar otros números de 1 a 3 dígitos
    texto = re.sub(r'\b(?!19|20)\d{1,3}\b', '', texto)  # Conservar años y eliminar otros números

    # 3. Eliminar URLs y caracteres especiales (como puntuación), pero conservar palabras y fechas
    texto = re.sub(r'[^\w\s]https?://\S+|www\.\S+', ' ', texto)

    # 4. Eliminar espacios extras (múltiples espacios)
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto

# Aplicar la limpieza
resoluciones_limpias = [limpiar_texto(res) for res in resoluciones]

# Mostrar el texto limpio (primeros 500 caracteres)
print(resoluciones_limpias)


##PREPROCESAMIENTO (Lematización y Stopwords)

In [ ]:
# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

# Descargar stopwords y recursos de lematización de NLTK
nltk.download('stopwords')
nltk.download('wordnet')

# Diccionario de sinónimos para normalizar términos
sinonimos = {
    "privados": "privado",
    "privadas": "privado"
}

# Función para reemplazar sinónimos en el texto
def reemplazar_sinonimos(texto):
    for key, value in sinonimos.items():
        texto = texto.replace(key, value)
    return texto

# Función para lematizar y eliminar stopwords usando spaCy
def lematizar_y_eliminar_stopwords(texto):
    doc = nlp(texto)
    lemas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(lemas)

# Función para eliminar palabras redundantes
def eliminar_palabras_redundantes(texto):
    palabras_redundantes = ['de', 'él', 'as', 'ción', 'sr', 'etc', 'co', 'sr']
    return ' '.join([palabra for palabra in texto.split() if palabra not in palabras_redundantes])

# Función para unificar plurales y singulares (lematización adicional)
def lematizar_palabras(vocabulario):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(palabra) for palabra in vocabulario.split()])

# Aplicar limpieza, lematización, normalización de sinónimos, eliminación de redundancias y unificación de plurales
def procesar_resoluciones(res):
    # Lematización y eliminación de stopwords con spaCy
    res_procesada = lematizar_y_eliminar_stopwords(res)
    # Reemplazo de sinónimos
    res_procesada = reemplazar_sinonimos(res_procesada)
    # Eliminación de palabras redundantes
    res_procesada = eliminar_palabras_redundantes(res_procesada)
    # Unificación de plurales y singulares
    res_procesada = lematizar_palabras(res_procesada)

    return res_procesada

# Aplicar todo el procesamiento a las resoluciones
resoluciones_procesadas = [procesar_resoluciones(res) for res in resoluciones]

# Mostrar el resultado de la primera resolución
#print(resoluciones_procesadas[0][:500])


##ANÁLISIS con TF-IDF

In [ ]:
# Inicializar el vectorizador TF-IDF sin eliminación de stopwords (ya fueron eliminadas previamente)
vectorizer = TfidfVectorizer(max_features=150, ngram_range=(3,3))

# Aplicar TF-IDF a las resoluciones procesadas
X = vectorizer.fit_transform(resoluciones_procesadas)

# Convertir el resultado a un DataFrame para visualizar las palabras y su peso
df_tfidf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Mostrar el resultado
print(df_tfidf.head())


In [ ]:
palabras_importantes = df_tfidf.sum().sort_values(ascending=False)
print(palabras_importantes.head(20))  # Muestra las 10 palabras más importantes


In [ ]:
import matplotlib.pyplot as plt
palabras_importantes.head(20).plot(kind='bar')
plt.show()


##VISUALIZACIÓN de Resultados

In [ ]:
# Definir un umbral para las palabras clave
umbral = 0.1  # Ajusta según sea necesario

# Crear un diccionario para almacenar los temas
temas = {}

# Agrupar palabras clave por temas
for i, row in df_tfidf.iterrows():
    palabras_importantes = row[row > umbral].index.tolist()  # Obtener palabras clave con TF-IDF > umbral
    for palabra in palabras_importantes:
        if palabra not in temas:
            temas[palabra] = []
        temas[palabra].append(i)  # Agregar el índice de la resolución al tema correspondiente

# Crear una nube de palabras para las palabras importantes
texto_palabras_importantes = ' '.join(temas.keys())  # Unir las palabras en un solo string

# Crear la nube de palabras
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto_palabras_importantes)

# Visualizar la nube de palabras
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de Palabras Importantes')
plt.show()


In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px
import pandas as pd
from gensim.models import Word2Vec

# Suponiendo que resoluciones_procesadas es una lista de textos procesados donde cada texto es una lista de palabras
resoluciones_tokenizadas = [resolucion.split() for resolucion in resoluciones_procesadas]

# Entrenar el modelo Word2Vec
modelo_w2v = Word2Vec(sentences=resoluciones_tokenizadas, vector_size=100, window=5, min_count=2, workers=4)

# Obtener el vocabulario del modelo
palabras_w2v = list(modelo_w2v.wv.index_to_key)

# Obtener los embeddings de las palabras
X = modelo_w2v.wv[palabras_w2v]

# Reducir la dimensionalidad a 2D usando t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=5, n_iter=3000)
result_tsne = tsne.fit_transform(X)

# Crear un DataFrame para usar con Plotly
df = pd.DataFrame(result_tsne, columns=['x', 'y'])
df['palabra'] = palabras_w2v

# Crear gráfico interactivo con Plotly
fig = px.scatter(df, x='x', y='y', text='palabra')

# Mostrar las palabras al pasar el cursor por encima de los puntos
fig.update_traces(textposition='top center', hovertemplate='<b>%{text}</b>')

# Mostrar el gráfico interactivo
fig.show()
